#Pet Image Segementation Using Modified U-Nets built on the Oxford IIIT Pets Dataset

Image Segementation is the process on taking an image as an input and indivually labling if each pixel is part of the object, bording the object, or is not part of the object. The Oxford IIIT Pets Dataset is perfect for this. It is a 37 category database with around 200 images in each category. We can further increase this with data augmentation. All of the images have a corresponding mask which has all the pixels divided into 3 classes: on the pet, bordering the pet, or outside the pet. Using this, we can train a modified U-Net to predict these masks when faced with new images. This model has acheived a 92% accuracy on the validation data, which is very high considering that we can further improve this with more epoches and model tuning.

We must first begin by importing the nessesary libaries into our program. We will be using TensorFlow and Keras to build and train the model, MatPlotLib to show our images and masks, and TensorFlow Datasets to access our dataset.

##Preparing Our Data To Be Processed

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix
import matplotlib.pyplot as plt


Now that we have imported all of our libraries, we can start to load the dataset and get ready for the data to be processed.

In [3]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Shuffling /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incomplete9F31AI/oxford_iiit_pet-train.tfrecord*...:…

Generating test examples...:   0%|          | 0/3669 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_iiit_pet/3.2.0.incomplete9F31AI/oxford_iiit_pet-test.tfrecord*...: …

Dataset oxford_iiit_pet downloaded and prepared to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0. Subsequent calls will reuse this data.


Next, we will make a load_image function that will resize the image and the mask to (128, 128). We will also make a function to normizlize the image, which will reduce the value range of the image from (0, 255) to (0, 1).

In [4]:
def load_image(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128), method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  input_image, input_mask = normalize_image(input_image, input_mask)
  return input_image, input_mask

def normalize_image(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

We will also now need to add our data augmentation. This will be impletemented in the form of a class and will randomly flip our images and masks using a set seed.

In [8]:
class Augment(tf.keras.layers.Layer):
  def __init__(self, seed=42):
    super().__init__()
    self.augment_inputs = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)
    self.augment_labels = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)

  def call(self, inputs, labels):
    inputs = self.augment_inputs(inputs)
    labels = self.augment_labels(labels)
    return inputs, labels

Now we can group our images into batches to be processed.

In [9]:
train_images = dataset['train'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
test_images = dataset['test'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

train_batches = (train_images.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat().map(Augment()).prefetch(buffer_size=tf.data.AUTOTUNE))
test_batches = test_images.batch(BATCH_SIZE)

##Creating and Training the Model

First we will begin by creating our base model. It will be based on the pretrained MobileNetV2 model. We will also create the second model, which will be based on pix2pix.


In [11]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

layer_names = [
    'block_1_expand_relu',
    'block_3_expand_relu',
    'block_6_expand_relu',
    'block_13_expand_relu',
    'block_16_project',
]
base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

down_stack.trainable = False

up_stack = [pix2pix.upsample(512, 3), pix2pix.upsample(256, 3), pix2pix.upsample(128, 3), pix2pix.upsample(64, 3),]


Now we can define the function that will make our U-Net Model.

In [12]:
def U_net_model(output_channels:int, down_stack, up_stack):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  skips = down_stack(inputs)
  outputs = skips[-1]
  skips = reversed(skips[:-1])

  for up, skip in zip(up_stack, skips):
    outputs = up(outputs)
    concatenate = tf.keras.layers.Concatenate()
    outputs = concatenate([outputs, skip])

  last = tf.keras.layers.Conv2DTranspose(filters=output_channels, kernel_size=3, strides=2, padding='same')
  outputs = last(outputs)
  return tf.keras.Model(inputs=inputs, outputs=outputs)

Now we can finally create and compile the model.

In [ ]:
OUTPUT_CLASSES = 3

model = U_net_model(OUTPUT_CLASSES, down_stack, up_stack)
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

It is now time to train the model we have just built using the train and test batches.

In [ ]:
EPOCHS = 20
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model.fit(train_batches, epochs=EPOCHS, steps_per_epoch=STEPS_PER_EPOCH, validation_steps=VALIDATION_STEPS, validation_data=test_batches)

Now we can save the model in three different formats for later use. We will be saving it in the legacy .h5 format, the new .keras format, and the TensorFlow SavedModel.

In [ ]:
model.save("pets.h5")
model.save("pets.keras")
model.save("model/dogs")

##Predict Using the Model

Now we will use our model to predict a mask against new data. But first, we must load the model off the disk.

In [ ]:
def attempt_load(i):
  try:
    model = tf.keras.models.load_model('pets'+names[i])
    return model
  except:
    attempt_load(i+1)

names = ['.keras', '', '.h5']

model = attempt_load(0)

Now we need to create our predicted mask.

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.math.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

We also need a function that can display the image and the predicted mask.

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))
  titles = ['Input Image', 'Predicted Mask']
  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(titles[i])
    plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

Finally, we need a way to get the users' picture and use our model to predict the mask for that picture.

In [ ]:
def show_predictions(image_url, model):
  image = tf.keras.utils.get_file(origin=image_url)
  image = tf.keras.utils.load_img(image)
  image = tf.keras.utils.img_to_array(image)
  image = tf.image.resize(image, (128,128))
  image = tf.cast(image, tf.float32) / 255.0
  image = tf.expand_dims(image, axis=0)
  pred_mask = model.predict(image)
  display([image[0], create_mask(pred_mask)])

And now lets wrap it all together!

In [ ]:
while True:
  url = input("Please enter an image url:")
  try:
    image = tf.keras.utils.get_file(origin=url)
    image = tf.keras.utils.load_img(image)
    break
  except:
    print("That is not a valid link")

show_predictions(url, model)